# AE Algorithms and AE class

As explained in section 2.5 of notebook *02_AmplitudeEstimationBTC* one of the main ingredients of the **AE BTC** is the **AE** algorithm used. In the present code implementation, 6 different algorithms can be used and all of them are managed by the *AE* class of the *BTC_02_AE/QQuantLib/AE/ae_class* module. The present notebook offers insight into the different **AE** algorithms and the keys of the input dictionary of the *AE* class for configuring them.

As remaining the **AE kernel** is given a unitary operator $\mathbf{A(f_{x_i})}$ such that:

$$|\Psi\rangle= \mathbf{A}|0\rangle_n  = \sqrt{a} |\Psi_0\rangle  + \sqrt{1-a}|\Psi_1\rangle \tag{1}$$

The **AE kernel** tries to get an estimation of the probability of obtaining the state $|\Psi_0\rangle$, this is an estimator of $a$.

In the following cells we create the operator $\mathbf{A(f_{x_i})}$ that we are going to use for testing the different algorithms.

In [ ]:
import numpy as np

def sin_integral(a,b):
    return np.cos(a)-np.cos(b)

start = [0.0, np.pi]
end = [3.0*np.pi/8.0, 5.0*np.pi/4.0]

#First integration domain
a = start[0]
b = end[0]
#For fix the number of discretization intervals
n=4
domain_x = np.linspace(a, b, 2**n+1)

#The selected fucntion
f = np.sin

#Discretization of the selected function
f_x = []
x_ = []
for i in range(1, len(domain_x)):
    step_f = (f(domain_x[i]) + f(domain_x[i-1]))/2.0
    #print(i)
    f_x.append(step_f)
    x_.append((domain_x[i] + domain_x[i-1])/2.0)
f_x = np.array(f_x)
x_ = np.array(x_)

normalization = np.max(np.abs(f_x))
print("Normalization constant: {}".format(normalization))
#normalization = 1.0
f_norm_x = f_x/normalization

Riemann = (np.sum(f_x)*(b-a))/2**n

print("Riemann sum integral: {}".format(Riemann))

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
from QQuantLib.DL.encoding_protocols import Encoding

In [ ]:
encoding_object = Encoding(
    array_function=f_norm_x, 
    array_probability=None, 
    encoding=2
)
encoding_object.run()

## 1. MonteCarlo Amplitude Estimation (MCAE)

In this case only the unitary operator $\mathbf{A(f_{x_i})}$ that encodes the function to integrate is mandatory. the idea is to execute the circuit and measure the probability of obtaining the state $|\Psi_0\rangle$: $a$.

For this technique, the only mandatory input of the configuration dictionary of the **AE** class will be the number of shots for executing the circuit that is provided using the key: *shots*.

In [ ]:
sys.path.append("../../../")
from qpu.select_qpu import select_qpu
# List with the strings taht should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]

qpu_config = {
    #the following strings can be used:
    #
    "qpu_type": ideal_qpus[0], 
    # The following keys are used for configuring noisy simulations
    "t_gate_1qb" : None,
    "t_gate_2qbs" : None,
    "t_readout": None,
    "depol_channel" : {
        "active": False,
        "error_gate_1qb" : None,
        "error_gate_2qbs" : None
    },
    "idle" : {
        "amplitude_damping": False,
        "dephasing_channel": False,
        "t1" : None,
        "t2" : None
    },
    "meas": {
        "active":False,
        "readout_error": None
    }
}

qpu = select_qpu(qpu_config)

In [ ]:
from QQuantLib.AE.ae_class import AE

In [ ]:
mcae_dict = {
    #QPU is alwways mandatory
    'qpu': qpu,
    #shots
    'shots': 1000,
    'ae_type': 'MCAE'
}

In [ ]:
mcae = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **mcae_dict
)
# We need to execute run method for solving the AE problem
mcae.run()
# Estimation of the a
mcae_pdf = mcae.ae_pdf

The **MCAE** does not provide upper and limit bounds but can be easily computed using as an error: $$\frac{1}{\sqrt{n_{shots}}}$$ where $n_{shots}$ is the number of shots.

In [ ]:
mcae_pdf

In [ ]:
# Rieman estimation
(b-a)*normalization*(2**n*np.sqrt(mcae_pdf))/2**n

In [ ]:
Riemann

## 2. Classical Quantum Phase Estimation (CQPEAE).

This is the canonical Quantum Phase Estimation method presented in the notebook **01_AmplitudeEstimationKernel**. This algorithm needs the Grover operator of  $\mathbf{A(f_{x_i})}$, $\mathbf{G}(\mathbf{A(f_{x_i})})$. In this case, the configuration keys for the *AE* class are:
* *auxiliar_qbits_number*: number of auxiliary (or ancilla) qubits used for doing the QFT. This number of qubits sets the precision of the returned estimation.
* *shots*: number of shots for executing the complete circuit.

Reference paper:
* Brassard, Gilles and Hoyer, Peter and Mosca, Michele and Tapp, Alain (2000). Quantum Amplitude Amplification and Estimation. AMS Contemporary Mathematics Series, **305**.


In [ ]:
cqpeae_dict = {
    #QPU is alwways mandatory
    'qpu': qpu,
    #shots
    'shots': 100,
    # Number of ancilla qubits for QFT
    'auxiliar_qbits_number': 10,
    'ae_type': 'CQPEAE'
}

In [ ]:
cqpeae = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **cqpeae_dict
)
# We need to execute run method for solving the AE problem
cqpeae.run()
# Estimation of the a
cqpeae_pdf = cqpeae.ae_pdf

In [ ]:
cqpeae_pdf

In [ ]:
# Rieman estimation
(b-a)*normalization*(2**n*np.sqrt(cqpeae_pdf))/2**n

In [ ]:
Riemann

## 3. Maximum Likelihood Amplitude Estimation (MLAE)

This algorithm needs the Grover operator of  $\mathbf{A(f_{x_i})}$, $\mathbf{G}(\mathbf{A(f_{x_i})})$. It is an iterative algorithm. 

In each step of the algorithm, the following operation is executed as a circuit: $$\mathbf{G}^{m_k} \mathbf{A(f_{x_i})} |0\rangle_n$$ where $m_k$ is selected in advance and the circuit is measured a fixed number of shots ($n_k$) and the probability of obtaining the $|\Psi_0\rangle$ is obtained (h_k). 

Using the properties of the *Grover* operator it can be shown that the following equation holds:

$$\mathbf{G}^{m_k} \mathbf{A(f_{x_i})} |0\rangle_n  =\sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$

where $$\sqrt{a} = \sin \theta$$.

In each step of the algorithm, the likelihood of obtaining $h_k$ in the function of $\theta$ is obtained: $l(\theta|h_k)$. Then the different likelihoods are gathered into a a cost function

$$C(\theta) = -\log\left(\prod_{k = 0}^M l_k(\theta,h_k)\right)$$

Then using a classical optimisation program (by default the *AE* class will use **brute force** scipy optimization:  https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brute.html) we compute the $\theta^*$ that minimizes the cost function. Finally, the desired amplitude is computed by:  $$a=sin^2(\theta^*)$$

The configuration keys for the *AE* are:
* *schedule*: list of list where the different $m_k$ and $n_k$ should be provided.
* *delta*: float for configuring the *brute force* optimizer. To avoid problems in 0 and pi/2 theta limits for defining the search domain.
* *ns*: int for defining the number of grid points used by the *brute force optimizer*.

Reference paper:
* Yohichi Suzuki and Shumpei Uno and Rudy Raymond and Tomoki Tanaka and Tamiya Onodera and Naoki Yamamoto (2020). Amplitude estimation without phase estimation **19**(2).

In [ ]:
mk = [1, 2, 3, 4, 5]
nk = [100] * len(mk)
schedule = [mk, nk]

mlae_dict = {
    #QPU is alwways mandatory
    'qpu': qpu,
    #MLAE
    #MLAE schedule
    'schedule': schedule,
    'delta': 1.0e-8,
    'ns': 100000,
    'ae_type': 'MLAE'
}

In [ ]:
mlae = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **mlae_dict
)
# We need to execute run method for solving the AE problem
mlae.run()
# Estimation of the a
mlae_pdf = mlae.ae_pdf

In [ ]:
mlae_pdf

In [ ]:
# Rieman estimation
(b-a)*normalization*(2**n*np.sqrt(mlae_pdf))/2**n

In [ ]:
Riemann

## 4. Iterative Quantum Amplitude Estimation (IQAE)

This algorithm needs the Grover operator of  $\mathbf{A(f_{x_i})}$, $\mathbf{G}(\mathbf{A(f_{x_i})})$. It is an iterative algorithm.

In each step of the algorithm the property of the *Grover* operator: $$\mathbf{G}^{m_k} \mathbf{A(f_{x_i})} |0\rangle_n  =\sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$ is used for obtaining some close bounds for the estimation of $a$.

The **IQAE** needs an error $\epsilon$ and a confident interval $\alpha$ as input and the final mission is estimating some upper and lower bounds  $(a_l, a_u)$ such $a$ satisfies that: $$P\big[a \in [a_l, a_u]\big] \gt 1-\alpha$$ and $$\frac{a_u-a_l}{2} \leq \epsilon$$

The configuration keys for the *AE* are:
* *epsilon*: float fot the $\epsilon$ of the **IQAE** algorithm.
* *alpha*: float confidence level $\alpha$ of the **IQAE** algorithm.
* *shots*: number of shots

Reference paper:
* Dmitry Grinko and Julien Gacon and Christa Zoufal and Stefan Woerner (2021). Iterative quantum amplitude estimation, npj Quantum Information **7**(1)

In [ ]:
iqae_dict = {
    #QPU is alwways mandatory
    'qpu': qpu,
    #MLAE
    #MLAE schedule
    'epsilon' : 0.001,
    'alpha': 0.05,
    'shots': 1000,
    'ae_type': 'IQAE'
}

In [ ]:
iqae = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **iqae_dict
)
# We need to execute run method for solving the AE problem
iqae.run()
# Estimation of the a
iqae_pdf = iqae.ae_pdf

In [ ]:
iqae_pdf

In [ ]:
# Rieman estimation
(b-a)*normalization*(2**n*np.sqrt(iqae_pdf))/2**n

In [ ]:
Riemann

# 5. Real Quantum Amplitude Estimation (RQAE).

This algorithm needs the Grover operator of  $\mathbf{A(f_{x_i})}$, $\mathbf{G}(\mathbf{A(f_{x_i})})$. It is an iterative algorithm. It computes the amplitude $a$ so it can return negative values. 

The **RQAE** needs an error $\epsilon$ and a confident interval $\gamma$ as input and the final mission is estimating some upper and lower bounds  $(a_l, a_u)$ such $a$ satisfies that: $$P\big[a \in [a_l, a_u]\big] \gt 1-\gamma$$ and $$\frac{a_u-a_l}{2} \leq \epsilon$$

The configuration keys for the *AE* are:
* *epsilon*: float fot the $\epsilon$ of the **RQAE** algorithm.
* *gamma*: float confidence level $\gamma$ of the **RQAE** algorithm.
* *ratio*: the **RQAE** allows the user to set the amplification ratio (this sets the times that the *Grover* operator is applied in each step). The recommended value is 2.

Reference paper:
* Manzano, Alberto and Musso, Daniele and Leitao, Álvaro (2023). Real Quantum Amplitude Estimation, EPJ Quantum Technology, **10**.


In [ ]:
rqae_dict = {
    #QPU is alwways mandatory
    'qpu': qpu,
    #MLAE
    #MLAE schedule
    'epsilon' : 0.001,
    'gamma': 0.05,
    'q': 2.0,
    'ae_type': 'RQAE'
}

In [ ]:
rqae = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **rqae_dict
)
# We need to execute run method for solving the AE problem
rqae.run()
# Estimation of the a
rqae_pdf = rqae.ae_pdf

In [ ]:
rqae_pdf

In [ ]:
(b-a)*normalization*2**n*rqae_pdf/2**n

In [ ]:
Riemann

## 6. IQPEAE

The **IQPEAE** is a variant of the **CQPEAE** algorithm where the Quantum Fourier Transform is done using only an additional qubit. A lower number of qubits are used but the depth of the circuits is much higher. This algorithm needs the Grover operator of \mathbf{A(f_{x_i})}$, $\mathbf{G}(\mathbf{A(f_{x_i})})$.  The configuration keys for the *AE* class are:

* *cbits_number*: number of classical bits used for computing the returned estimation.
* *shots*: number of shots for executing the complete circuit.

Reference paper:

    Alexei Y. Kitaev (1995). Quantum measurements and the Abelian Stabilizer Problem. Electron. Colloquium Comput. Complex TR96.
    
**BE AWARE**

The IQPEAE needs a lot of computational resources for its execution. So the configuration and the code for the execution are provided as a Markdown notebook cell. For executing transform the cell to code but the computation time will be high!


iqpeae_dict = {
    #QPU is alwways mandatory
    'qpu': qpu,
    #shots
    'shots': 100,
    # Number of classical qubits for QFT
    'cbits_number': 10,
    'ae_type': 'IQPEAE'
}

iqpeae = AE(
    oracle=encoding_object.oracle,
    target=encoding_object.target,
    index=encoding_object.index,
    **iqpeae_dict
)
# We need to execute run method for solving the AE problem
iqpeae.run()
# Estimation of the a
iqpeae_pdf = iqpeae.ae_pdf
# Rieman estimation
print((b-a)*normalization*(2**n*np.sqrt(iqae_pdf))/2**n)
print(Riemann)

## 7. Other AE algorithms

As explained before, the **QQuantLib** library was developed in the framework of the **Financial Applications** case of the **WP 5** of the **NEASQC** project. In addition to the **AE** algorithms presented in this notebook, several improvements and modifications were made to them. All these modifications can be used too by providing the corresponding string to the *ae_type* key of the configuration dictionary. Until 2024/05 the **AE** algorithm modifications included were:

* Modified Iterative Quantum Amplitude Estimation (**MIQAE**). Modification of the **IQAE** that allows a better asymptotic behaviour.
* Modified Real Quantum Amplitude Estimation (**MRQAE**). Modification of the **IQAE** that allows a better asymptotic behaviour.
* Shots Real Quantum Amplitude Estimation (**sRQAE**). Modification of the **RQAE** where the user can provide the number of shots for each iteration of the algorithm. This modification usually provides a better experimental performance.
* Extended Real Quantum Amplitude Estimation (**eRQAE**). Modification of the **RQAE** where the user can provide a schedule for guiding the evolution of the algorithm.

For more information about these modifications please review the notebooks:

* *misc/notebooks/05_Iterative_Quantum_Phase_Estimation_Class.ipynb*
* *misc/notebooks/07-02_Improvements_on_Real_Quantum_Amplitude_Estimation.ipynb*

Of the **FinancialApplications** software package (https://github.com/NEASQC/FinancialApplications)